In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

In [2]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

In [3]:
def booster_version(table_cells):
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out

In [4]:
def landing_status(table_cells):
    out = [i for i in table_cells.strings][0]
    return out

In [5]:
def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass

In [6]:
def extract_column_from_header(row):
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    column_name = ' '.join(row.contents)
    if not column_name.strip().isdigit():
        column_name = column_name.strip()
    return column_name

In [7]:
# Wikipedia snapshot URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Request HTML page
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract tables
html_tables = soup.find_all('table', {"class": "wikitable plainrowheaders collapsible"})
first_launch_table = html_tables[2]

# Extract column names
column_names = []
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)


In [8]:
# Create dictionary for DataFrame
launch_dict = dict.fromkeys(column_names)
del launch_dict['Date and time ( )']
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [10]:
# Extract table rows
extracted_row = 0

for table_number, table in enumerate(html_tables):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
            else:
                flag = False
        
        row = rows.find_all('td')

        # Ensure row has enough elements to prevent IndexError
        if flag and len(row) >= 9:
            extracted_row += 1
            launch_dict['Flight No.'].append(flight_number)
            
            # Extract Date and Time
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(',') if len(datatimelist) > 0 else 'N/A')
            launch_dict['Time'].append(datatimelist[1] if len(datatimelist) > 1 else 'N/A')

            # Extract Booster Version
            bv = booster_version(row[1]) if len(row) > 1 else 'N/A'
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Extract Other Data
            launch_dict['Launch site'].append(row[2].a.string if len(row) > 2 and row[2].a else 'N/A')
            launch_dict['Payload'].append(row[3].a.string if len(row) > 3 and row[3].a else 'N/A')
            launch_dict['Payload mass'].append(get_mass(row[4]) if len(row) > 4 else 'N/A')
            launch_dict['Orbit'].append(row[5].a.string if len(row) > 5 and row[5].a else 'N/A')
            launch_dict['Customer'].append(row[6].a.string if len(row) > 6 and row[6].a else 'N/A')
            launch_dict['Launch outcome'].append(list(row[7].strings)[0] if len(row) > 7 else 'N/A')
            launch_dict['Booster landing'].append(landing_status(row[8]) if len(row) > 8 else 'N/A')

# Convert dictionary to DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

# Save DataFrame to CSV
df.to_csv('spacex_web_scraped.csv', index=False)


In [11]:
# Create DataFrame
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

In [12]:
# Save DataFrame to CSV
df.to_csv('spacex_web_scraped.csv', index=False)
print("CSV file 'spacex_web_scraped.csv' has been successfully created.")

CSV file 'spacex_web_scraped.csv' has been successfully created.
